In [2]:
import os
import sys

os.environ["PYLIB"]=os.path.join(os.environ["SPARK_HOME"],'python','lib')
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'py4j-0.10.1-src.zip'))
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'pyspark.zip'))

import pyspark
myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession.builder\
    .master("local")\
    .appName("myApp")\
    .config("spark.sql.warehouse.dir", "C:/Users/G312")\
    .getOrCreate()

In [ ]:
#파일 내려받고 읽기

In [9]:
import os
_trainDf = spark.read.format('com.databricks.spark.csv')\
    .options(header='true', inferschema='true')\
    .load(os.path.join("data","kaggle","titanic","train.csv"))

In [23]:
_trainDf.show(3, truncate=True)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-----------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|testOrtrain|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-----------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25|     |       S|      train|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|      train|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925|     |       S|      train|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-----------+
only showing top 3 rows



In [15]:
_testDf = spark.read.format('com.databricks.spark.csv')\
    .options(header='true', inferschema='true')\
    .load(os.path.join("data","kaggle","titanic","test.csv"))

_testDf.show(3, truncate=True)

+-----------+------+--------------------+------+----+-----+-----+------+------+-----+--------+
|PassengerId|Pclass|                Name|   Sex| Age|SibSp|Parch|Ticket|  Fare|Cabin|Embarked|
+-----------+------+--------------------+------+----+-----+-----+------+------+-----+--------+
|        892|     3|    Kelly, Mr. James|  male|34.5|    0|    0|330911|7.8292|     |       Q|
|        893|     3|Wilkes, Mrs. Jame...|female|47.0|    1|    0|363272|   7.0|     |       S|
|        894|     2|Myles, Mr. Thomas...|  male|62.0|    0|    0|240276|9.6875|     |       Q|
+-----------+------+--------------------+------+----+-----+-----+------+------+-----+--------+
only showing top 3 rows



In [ ]:
#train/test 파일 합치기

In [16]:
from pyspark.sql.functions import lit, col
_trainDf = _trainDf.withColumn('testOrtrain',lit('train'))
_testDf = _testDf.withColumn('testOrtrain',lit('test'))

In [17]:
_testDf = _testDf.withColumn('Survived',lit(99))

In [18]:
_trainDf.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- testOrtrain: string (nullable = false)



In [19]:
_testDf.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- testOrtrain: string (nullable = false)
 |-- Survived: integer (nullable = false)



In [20]:
for c in _trainDf.columns:
    print c,

PassengerId Survived Pclass Name Sex Age SibSp Parch Ticket Fare Cabin Embarked testOrtrain


In [21]:
df=_trainDf.select('PassengerId','Survived','Pclass','Name','Sex','Age',\
                   'SibSp','Parch','Ticket','Fare','Cabin','Embarked','testOrtrain')\
            .union(_testDf.select('PassengerId','Survived','Pclass','Name','Sex','Age',\
                   'SibSp','Parch','Ticket','Fare','Cabin','Embarked','testOrtrain'))

In [ ]:
#Test/Train 선택

In [22]:
df.select('testOrtrain','Survived','Name')\
    .filter(df['testOrtrain']=='test').show(10)

+-----------+--------+--------------------+
|testOrtrain|Survived|                Name|
+-----------+--------+--------------------+
|       test|      99|    Kelly, Mr. James|
|       test|      99|Wilkes, Mrs. Jame...|
|       test|      99|Myles, Mr. Thomas...|
|       test|      99|    Wirz, Mr. Albert|
|       test|      99|Hirvonen, Mrs. Al...|
|       test|      99|Svensson, Mr. Joh...|
|       test|      99|Connolly, Miss. Kate|
|       test|      99|Caldwell, Mr. Alb...|
|       test|      99|Abrahim, Mrs. Jos...|
|       test|      99|Davies, Mr. John ...|
+-----------+--------+--------------------+
only showing top 10 rows



In [24]:
df.groupBy(df.testOrtrain).count().show()

+-----------+-----+
|testOrtrain|count|
+-----------+-----+
|      train|  891|
|       test|  418|
+-----------+-----+



In [25]:
#데이터 변환

In [26]:
##Missing 데이터의 처리
from pyspark.sql.functions import count
df.agg(*[count(c).alias(c) for c in df.columns]).show() #age와 fare 열에 결측값이 존재

+-----------+--------+------+----+----+----+-----+-----+------+----+-----+--------+-----------+
|PassengerId|Survived|Pclass|Name| Sex| Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|testOrtrain|
+-----------+--------+------+----+----+----+-----+-----+------+----+-----+--------+-----------+
|       1309|    1309|  1309|1309|1309|1046| 1309| 1309|  1309|1308| 1309|    1309|       1309|
+-----------+--------+------+----+----+----+-----+-----+------+----+-----+--------+-----------+



In [27]:
def countNull(df,var):
    return df.where(df[var].isNull()).count() #isNull로 결측값 확인

missing = {c: countNull(df,c) for c in ['Survived','Age','SibSp','Parch','Fare']}

print missing

{'Fare': 1, 'Age': 263, 'SibSp': 0, 'Survived': 0, 'Parch': 0}


In [28]:
print df.filter("Age is NULL").show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+------+------+-----+--------+-----------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|Ticket|  Fare|Cabin|Embarked|testOrtrain|
+-----------+--------+------+--------------------+------+----+-----+-----+------+------+-----+--------+-----------+
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|330877|8.4583|     |       Q|      train|
|         18|       1|     2|Williams, Mr. Cha...|  male|null|    0|    0|244373|  13.0|     |       S|      train|
|         20|       1|     3|Masselmani, Mrs. ...|female|null|    0|    0|  2649| 7.225|     |       C|      train|
|         27|       0|     3|Emir, Mr. Farred ...|  male|null|    0|    0|  2631| 7.225|     |       C|      train|
|         29|       1|     3|"O'Dwyer, Miss. E...|female|null|    0|    0|330959|7.8792|     |       Q|      train|
+-----------+--------+------+--------------------+------+----+-----+----

In [29]:
print df.filter("Fare is NULL").show(5)

+-----------+--------+------+------------------+----+----+-----+-----+------+----+-----+--------+-----------+
|PassengerId|Survived|Pclass|              Name| Sex| Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|testOrtrain|
+-----------+--------+------+------------------+----+----+-----+-----+------+----+-----+--------+-----------+
|       1044|      99|     3|Storey, Mr. Thomas|male|60.5|    0|    0|  3701|null|     |       S|       test|
+-----------+--------+------+------------------+----+----+-----+-----+------+----+-----+--------+-----------+

None


In [32]:
from pyspark.sql import functions as F #age와 Fare의 평균갑 구하기

avgAge=df.agg(F.avg(df['Age']).alias('meanAge')).collect()
avgFare=df.agg(F.avg(df['Fare']).alias('meanFare')).collect()

print avgAge[0]['meanAge']
print avgFare[0]['meanFare']

print df.groupBy().mean('Age').first()
print df.groupBy().mean('Fare').first()

29.8811376673
33.2954792813
Row(avg(Age)=29.881137667304014)
Row(avg(Fare)=33.29547928134553)


In [33]:
df.describe(['Age']).show() #age열에 대한 통계

+-------+------------------+
|summary|               Age|
+-------+------------------+
|  count|              1046|
|   mean|29.881137667304014|
| stddev| 14.41349321127132|
|    min|              0.17|
|    max|              80.0|
+-------+------------------+



In [34]:
my = spark.createDataFrame([[2019], [2020], [2021], [None]], ['year'])
my = my.fillna({'year':'9999'}) #Null 값을 대체하는 fillna 함수
my.show()

+----+
|year|
+----+
|2019|
|2020|
|2021|
|9999|
+----+



In [35]:
from pyspark.sql.functions import when,isnull #isnull, when을 통해 결측값을 평균으로 대체
df=df.withColumn("Age", when(isnull(df['Age']), avgAge[0]['meanAge']).otherwise(df.Age))
df=df.withColumn("Fare", when(isnull(df['Fare']), avgFare[0]['meanFare']).otherwise(df.Fare))
#df.show(10)

In [36]:
#Outlier

In [37]:
quantiles=df.stat.approxQuantile('Age', [0.25,0.75], 0.0 )
print quantiles

[22.0, 35.0]


In [38]:
mid50 = quantiles[1]-quantiles[0]
lower = quantiles[0] - 1.15*mid50
upper = quantiles[1] + 2.0*mid50
print lower, upper

7.05 61.0


In [39]:
df.filter((df.Age<lower) | (df.Age>upper)).show(3,truncate=True)

+-----------+--------+------+--------------------+------+---+-----+-----+-------+------+-----+--------+-----------+
|PassengerId|Survived|Pclass|                Name|   Sex|Age|SibSp|Parch| Ticket|  Fare|Cabin|Embarked|testOrtrain|
+-----------+--------+------+--------------------+------+---+-----+-----+-------+------+-----+--------+-----------+
|          8|       0|     3|Palsson, Master. ...|  male|2.0|    3|    1| 349909|21.075|     |       S|      train|
|         11|       1|     3|Sandstrom, Miss. ...|female|4.0|    1|    1|PP 9549|  16.7|   G6|       S|      train|
|         17|       0|     3|Rice, Master. Eugene|  male|2.0|    4|    1| 382652|29.125|     |       Q|      train|
+-----------+--------+------+--------------------+------+---+-----+-----+-------+------+-----+--------+-----------+
only showing top 3 rows



In [40]:
#성별 구분
df.groupBy('Sex').count().show()

+------+-----+
|   Sex|count|
+------+-----+
|female|  466|
|  male|  843|
+------+-----+



In [41]:
import re
def getTitle(name):
    title=None
    if re.search(".*Mr\..*", name): #남자 Mr
        title="male"
    elif re.search(".*[Miss|Mrs|Ms]\..*", name): #여자 Miss Mrs Ms
        title="female"
    return title

In [42]:
names=["Braund, Mr. Owen Harris",
       "Cumings, Mrs. John Bradley (Florence Briggs Thayer)",
       "Heikkinen, Miss. Laina",
       "Ms.hello",
       "No title"]
for n in names:
    print getTitle(n)

male
female
female
female
None


In [43]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

getTitleUdf = udf(getTitle, StringType())
df = df.withColumn('Title', getTitleUdf(df['Name']))

In [44]:
df.select('testOrtrain','Name','Title','Sex')\
    .filter(df['testOrtrain']=='test')\
    .show(10,truncate=False)

+-----------+--------------------------------------------+------+------+
|testOrtrain|Name                                        |Title |Sex   |
+-----------+--------------------------------------------+------+------+
|test       |Kelly, Mr. James                            |male  |male  |
|test       |Wilkes, Mrs. James (Ellen Needs)            |female|female|
|test       |Myles, Mr. Thomas Francis                   |male  |male  |
|test       |Wirz, Mr. Albert                            |male  |male  |
|test       |Hirvonen, Mrs. Alexander (Helga E Lindqvist)|female|female|
|test       |Svensson, Mr. Johan Cervin                  |male  |male  |
|test       |Connolly, Miss. Kate                        |female|female|
|test       |Caldwell, Mr. Albert Francis                |male  |male  |
|test       |Abrahim, Mrs. Joseph (Sophie Halaut Easu)   |female|female|
|test       |Davies, Mr. John Samuel                     |male  |male  |
+-----------+--------------------------------------

In [45]:
df.groupBy('Title').count().show()
df.groupBy('Sex').count().show()

+------+-----+
| Title|count|
+------+-----+
|  null|   19|
|female|  533|
|  male|  757|
+------+-----+

+------+-----+
|   Sex|count|
+------+-----+
|female|  466|
|  male|  843|
+------+-----+



In [46]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- testOrtrain: string (nullable = false)
 |-- Title: string (nullable = true)



In [47]:
#Label Features

In [48]:
df=df.withColumn("SurvivedD",_trainDf['Survived']\
    .cast("double"))\
    .drop('Survived')

df.groupBy('SurvivedD').count().show()

+---------+-----+
|SurvivedD|count|
+---------+-----+
|      0.0|  549|
|      1.0|  342|
|     99.0|  418|
+---------+-----+



In [49]:
df.show(5, truncate=True)

+-----------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-----------+------+---------+
|PassengerId|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|testOrtrain| Title|SurvivedD|
+-----------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-----------+------+---------+
|          1|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25|     |       S|      train|  male|      0.0|
|          2|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|      train|female|      1.0|
|          3|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925|     |       S|      train|female|      1.0|
|          4|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|      train|female|      1.0|
|          5|     3|Allen, Mr. Willia...|  male|35.0|  

In [50]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
#from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

SexIndexer = StringIndexer(inputCol="Sex", outputCol="SexI")
EmbarkedIndexer = StringIndexer(inputCol="Embarked", outputCol="EmbarkedI")

#PclassIndexer = StringIndexer(inputCol="Pclass", outputCol="SexI")

In [51]:
va = VectorAssembler(inputCols=["Pclass","SexI","Age","SibSp","Parch",\
                                "Fare","EmbarkedI"],\
                     outputCol="features")

In [52]:
pipeline = Pipeline(stages=[SexIndexer,EmbarkedIndexer,va])
model = pipeline.fit(df)

In [53]:
myDf = model.transform(df)

In [54]:
myDf.select('SurvivedD','features').show(10)

+---------+--------------------+
|SurvivedD|            features|
+---------+--------------------+
|      0.0|[3.0,0.0,22.0,1.0...|
|      1.0|[1.0,1.0,38.0,1.0...|
|      1.0|[3.0,1.0,26.0,0.0...|
|      1.0|[1.0,1.0,35.0,1.0...|
|      0.0|(7,[0,2,5],[3.0,3...|
|      0.0|[3.0,0.0,29.88113...|
|      0.0|(7,[0,2,5],[1.0,5...|
|      0.0|[3.0,0.0,2.0,3.0,...|
|      1.0|[3.0,1.0,27.0,0.0...|
|      1.0|[2.0,1.0,14.0,1.0...|
+---------+--------------------+
only showing top 10 rows



In [55]:
print "all num of rows: ",myDf.count()

all num of rows:  1309


In [56]:
train=myDf.filter(myDf['testOrtrain']=='train')
testDf=myDf.filter(myDf['testOrtrain']=='test')

In [57]:
myDf.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- testOrtrain: string (nullable = false)
 |-- Title: string (nullable = true)
 |-- SurvivedD: double (nullable = true)
 |-- SexI: double (nullable = true)
 |-- EmbarkedI: double (nullable = true)
 |-- features: vector (nullable = true)



In [58]:
train.count()

891

In [59]:
print 'test num of rows: ',testDf.count()

test num of rows:  418


In [60]:
trainDf,validateDf = train.randomSplit([0.7,0.3],seed=11) #검증/빌딩을 위해 두 그룹으로, 랜덤으로 나눈다.

In [67]:
trainDf.count()

628

In [66]:
 validateDf.count() #검증 DF

263

In [68]:
trainDf.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- testOrtrain: string (nullable = false)
 |-- Title: string (nullable = true)
 |-- SurvivedD: double (nullable = true)
 |-- SexI: double (nullable = true)
 |-- EmbarkedI: double (nullable = true)
 |-- features: vector (nullable = true)



In [69]:
trainDf.groupBy('SurvivedD').count().show() #여기부터 모델링

+---------+-----+
|SurvivedD|count|
+---------+-----+
|      0.0|  383|
|      1.0|  245|
+---------+-----+



In [70]:
from pyspark.ml.classification import LogisticRegression
 
# regPara: lasso regularisation parameter (L1)
lr = LogisticRegression().\
    setLabelCol('SurvivedD').\
    setFeaturesCol('features').\
    setRegParam(0.0).\
    setMaxIter(100).\
    setElasticNetParam(0.)

In [71]:
lrModel=lr.fit(trainDf)

In [72]:
lrDf = lrModel.transform(validateDf) #여기서부터 예측(predition/probability)

In [73]:
lrDf.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- testOrtrain: string (nullable = false)
 |-- Title: string (nullable = true)
 |-- SurvivedD: double (nullable = true)
 |-- SexI: double (nullable = true)
 |-- EmbarkedI: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = true)



In [74]:
lrDf.select('SurvivedD','rawPrediction','probability','prediction').show()

+---------+--------------------+--------------------+----------+
|SurvivedD|       rawPrediction|         probability|prediction|
+---------+--------------------+--------------------+----------+
|      1.0|[-0.7209575821080...|[0.32718215168700...|       1.0|
|      1.0|[-2.0503227895482...|[0.11401976925515...|       1.0|
|      1.0|[-1.9554360708280...|[0.12396181876919...|       1.0|
|      1.0|[-1.1467078201107...|[0.24109092679847...|       1.0|
|      1.0|[-1.4698503211520...|[0.18696536548151...|       1.0|
|      0.0|[1.69472254018706...|[0.84484421073581...|       0.0|
|      1.0|[1.26032179504787...|[0.77908149803135...|       0.0|
|      1.0|[-1.4501313540239...|[0.18998135132665...|       1.0|
|      1.0|[-0.0619658876835...|[0.48451348315217...|       1.0|
|      0.0|[2.56037173163627...|[0.92826721424349...|       0.0|
|      1.0|[-0.8920008140330...|[0.29069710229074...|       1.0|
|      0.0|[-1.2464028691324...|[0.22332344045916...|       1.0|
|      0.0|[1.95032687007

In [75]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator #여기서부터 평가

evaluator = BinaryClassificationEvaluator(rawPredictionCol = 'prediction',\
                                          labelCol='SurvivedD')

In [76]:
evaluator.evaluate(lrDf) #약 80퍼센트 -> 짐승보다 낫다

0.7919513103962241